In [1]:
import os
import glob
import base64
import pandas as pd
import ollama 
from tqdm import tqdm
import time
import json
from concurrent.futures import ProcessPoolExecutor, as_completed
import io
from PIL import Image

# --- 1. 配置 ---
# 数据集路径
IMAGE_DIR_1 = "data/HAM10000/HAM10000_images_part_1"
IMAGE_DIR_2 = "data/HAM10000/HAM10000_images_part_2"
METADATA_FILE = "data/HAM10000/HAM10000_metadata.csv"

# 结果和检查点路径
RESULTS_FILE = "ollama_native_classification_results.csv" # 建议换个新文件名以区分
CHECKPOINT_FILE = "ollama_native_checkpoint.json"       # 建议换个新文件名以区分

# 用于分类的类别列表
categories = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
categories_str = ", ".join(categories)

# 视觉语言模型的提示 (保持不变)
PROMPT_TEXT = f"""
你是一名世界级的皮肤科AI诊断助手。请按照以下步骤分析和分类提供的皮肤镜图像。

**第一步：特征分析（内心思考，不要输出）**
1.  **对称性**：病变的形状是否对称？
2.  **边缘**：边缘是清晰规则，还是模糊、不规则、有切迹？
3.  **颜色**：颜色是单一均匀，还是包含多种颜色（如棕、黑、红、蓝、白）？颜色分布是否均匀？
4.  **结构**：是否能观察到特定的皮肤镜结构？例如，色素网络、点状血管、树枝状血管、蓝白幕、乳头状结构等。
5.  **整体评估**：综合以上特征，病变给人的整体感觉是良性的（有序、规则）还是恶性的（混乱、不规则）？

**第二步：分类判断**
根据你的分析，将图像归类到以下七个类别之一：{categories_str}。
    - **黑色素瘤 (mel, Melanoma)** 特征：明显的不对称性、不规则边缘、颜色多样性（棕、黑、蓝、白、红等），  
      非典型色素网络、蓝白幕、放射状线条、负网状结构、不对称的小点或条纹、局部回避区等恶性特征。 
 
    - **基底细胞癌 (bcc, Basal Cell Carcinoma)** 特征：树枝状血管、蓝灰色卵圆巢、光滑珠光边缘、溃疡或结痂区域、车轮辐射状结构、白色条纹或亮点。 
 
    - **黑色素细胞痣 (nv, Melanocytic Nevus)** 特征：整体对称、规则的色素网络、均匀的棕色色调、清晰边界、  
      可见规则点状或球状结构、均匀分布的色素网格。 
 
    - **脂溢性角化病 (bkl, Benign Keratosis)** 特征：粉刺样开口、脑回状（丘脑状）结构、粘贴感外观、白色假网状结构、角质栓、黑点或伪毛囊口。 
 
    - **光化性角化病 (akiec, Actinic Keratosis)** 特征：红白交错的表面、毛细血管扩张、鳞屑、角质过度增生、淡棕或红色调，  
      可能可见“草地样”或“红白斑块状”结构。 
 
    - **皮肤纤维瘤 (df, Dermatofibroma)** 特征：中心棕色区伴周围淡色晕、放射状色素结构、中心瘢痕样白区、周边色素网络逐渐消退、轻微凹陷。 
 
    - **血管性病变 (vasc, Vascular Lesion)** 特征：均匀的红色至紫色区域、清晰可见的血管结构、点状或线状血管、湖状血管样分布、整体对称。

**第三步：输出结果**
请只输出最终确定的类别缩写。不要包含任何分析、解释或额外文字。
"""

# --- 2. 辅助函数 ---

def encode_image(image_path):
    """将图像文件编码为base64字符串。"""
    try:
        # 在原生 Ollama 库中，可以直接传递原始字节，但为了保持与多进程的兼容性，
        # 传递 base64 字符串仍然是一个好方法。
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"编码图像 {image_path} 时出错: {e}")
        return None

def classify_image(image_id, image_path):
    """
    使用Ollama模型对单个图像进行分类。
    返回一个元组 (image_id, result_dict)。
    """
    base64_image = encode_image(image_path)
    if not base64_image:
        return image_id, None

    try:
        # --- 2. 使用原生 ollama.chat 进行 API 调用 ---
        response = ollama.chat(
            model="qwen2.5vl:32b",
            messages=[
                {
                    'role': 'user',
                    'content': PROMPT_TEXT,
                }
            ],
            # 将图片作为独立的参数传递
            images=[base64_image],
            # 可以通过 options 传递参数，但对于简短回复，模型通常能遵循指令
            options={
                'num_predict': 10 
            }
        )
        
        # --- 3. 修改响应解析方式 ---
        predicted_class = response['message']['content'].strip().lower()

        # 验证模型的输出 (保持不变)
        if predicted_class not in categories:
            print(f"警告：图像 {image_id} 的模型返回了一个意外的类别 '{predicted_class}'。将其设置为'unknown'。")
            predicted_class = "unknown"

        return image_id, {"predicted_class": predicted_class}

    except Exception as e:
        print(f"分类图像 {image_id} 时出错: {e}")
        return image_id, None

def load_checkpoint():
    """从检查点文件加载已处理的图像和结果。"""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            data = json.load(f)
            return set(data.get("processed_images", [])), data.get("results", {})
    return set(), {}

def save_checkpoint(processed_images, results):
    """将已处理的图像和结果保存到检查点文件。"""
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump({"processed_images": list(processed_images), "results": results}, f, indent=4)


In [ ]:
print("--- 开始图像分类 (使用原生 Ollama) ---") # <--- 更新了打印信息
# 加载元数据
metadata_df = pd.read_csv(METADATA_FILE)
print(f"已加载 {len(metadata_df)} 张图像的元数据。")
# 获取所有图像路径
image_paths_part1 = glob.glob(os.path.join(IMAGE_DIR_1, '*.jpg'))
image_paths_part2 = glob.glob(os.path.join(IMAGE_DIR_2, '*.jpg'))
all_image_paths = image_paths_part1 + image_paths_part2

image_id_to_path = {os.path.basename(p).split('.')[0]: p for p in all_image_paths}
print(f"共找到 {len(all_image_paths)} 张图像。")
# 从检查点加载
processed_ids, results = load_checkpoint()
print(f"已加载检查点。{len(processed_ids)} 张图像已被处理。")
# 确定要处理的图像
unprocessed_tasks = []
for img_id in metadata_df['image_id']:
    if img_id not in processed_ids and img_id in image_id_to_path:
        unprocessed_tasks.append((img_id, image_id_to_path[img_id]))

print(f"找到 {len(unprocessed_tasks)} 张待处理的图像。")
if not unprocessed_tasks:
    print("没有新的图像需要处理。")
else:
    # 并行处理
    with ProcessPoolExecutor() as executor:
        # 为未处理的图像创建future
        futures = {executor.submit(classify_image, img_id, path): (img_id, path) for img_id, path in unprocessed_tasks}
        
        # 在任务完成时处理结果
        for future in tqdm(as_completed(futures), total=len(unprocessed_tasks), desc="正在分类图像"):
            image_id, result = future.result()
            if result:
                results[image_id] = result
                processed_ids.add(image_id)
                
                # 定期保存检查点（例如，每处理10张图像）
                if len(processed_ids) % 10 == 0:
                    save_checkpoint(processed_ids, results)
# 最后保存检查点和结果
print("分类完成。正在保存最终结果...")



In [ ]:
save_checkpoint(processed_ids, results)
# 将结果转换为DataFrame并保存为CSV
results_list = []
for image_id, result_data in results.items():
    results_list.append({
        "image_id": image_id,
        "predicted_class": result_data.get("predicted_class", "error")
    })

results_df = pd.DataFrame(results_list)
results_df.to_csv(RESULTS_FILE, index=False)
print(f"结果已保存到 {RESULTS_FILE}")
# --- 4. 评估 ---
print("\n--- 正在评估准确率 ---")
# 确保结果文件存在再进行评估
if not os.path.exists(RESULTS_FILE) or len(results_df) == 0:
    print("没有可评估的结果。")
    
merged_df = pd.merge(results_df, metadata_df[['image_id', 'dx']], on='image_id', how='inner')
merged_df = merged_df.rename(columns={'dx': 'true_class'})
correct_predictions = (merged_df['predicted_class'] == merged_df['true_class']).sum()
total_predictions = len(merged_df)
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"总评估预测数: {total_predictions}")
print(f"正确预测数: {correct_predictions}")
print(f"总体准确率: {accuracy:.4f}")
# 各类别准确率
if total_predictions > 0:
    class_accuracy = merged_df.groupby('true_class').apply(
        lambda x: (x['predicted_class'] == x['true_class']).mean()
    ).reset_index(name='accuracy')
    print("\n各类别准确率:")
    print(class_accuracy)